In [5]:
import pandas as pd
import numpy as np
import ssl
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import httplib2
ssl._create_default_https_context = ssl._create_unverified_context
response = urlopen('https://www.python.org')
import matplotlib.pyplot as plt

In [7]:
#create empty df with 24 episodes per season for 10 seasons
names = ['ross','chandler','joey','monica','phoebe','rachel']
names_5 = ['ross','joey','monica','phoebe','rachel']
seasons = [num if len(num)==2 else '0'+num for num in np.arange(1,11).astype(str) ]
episodes = [num if len(num)==2 else '0'+num for num in np.arange(1,25).astype(str) ]

df = pd.DataFrame(columns = names)
df['Season'],df['Episode'] = '',''
df['Season'] = seasons*len(episodes)
df['Episode'] = np.repeat(episodes,len(seasons))
df['Season'] = pd.to_numeric(df["Season"])
df['Episode'] = pd.to_numeric(df["Episode"])
df.sort_values(by=['Season', 'Episode'],inplace = True)
df.set_index(['Season','Episode'],inplace = True) #not sure if I want to keep this indexing

In [74]:
seasons = [num if len(num)==2 else '0'+num for num in np.arange(1,11).astype(str) ]
episodes = [num if len(num)==2 else '0'+num for num in np.arange(1,25).astype(str) ]

for season in seasons:
    print(season)
    for episode in episodes:
        url = "https://fangj.github.io/friends/season/"+season+episode+'.html'
        if season =='09' and episode == '23':
            url = 'https://fangj.github.io/friends/season/0923-0924.html'
        if season =='10' and episode =='17':
            url = 'https://fangj.github.io/friends/season/1017-1018.html'
        h = httplib2.Http()
        resp = h.request(url, 'HEAD')
        if int(resp[0]['status']) < 400:
            html = urlopen(url).read()
            soup = BeautifulSoup(html)
            # kill all script and style elements
            for script in soup(["script", "style"]):
                script.extract()    # rip it out
            # get text
            text = soup.get_text()
            # break into lines and remove leading and trailing space on each
            lines = (line.strip() for line in text.splitlines())
            # break multi-headlines into a line each
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            # drop blank lines
            text = '\n'.join(chunk for chunk in chunks if chunk)
            genqa(text)
            
#             

01
02
03
04
05
06
07
08
09
10


In [73]:
def genqa(text):

    f = open("all_lines.txt", "a+")
    q = ''
    a = ''
    p = ''
    aa = False 
    qq = False
    names_5 = ['ross','joey','monica','phoebe','rachel']
    for line in text.split('\n'):

        #if we find a new line with a colon, start either 'q' or 'a' string
        if len(re.findall(r'^.*:',line)) >0:

            #append previous strings if both q and a>0
            if len(a)>0 and len(q)>0:
                a = re.sub("[\(\[].*?[\)\]]", "", a)
                q = re.sub("[\(\[].*?[\)\]]", "", q)
                f.write(' '.join(['Q:',q,'\n']))
                f.write(' '.join(['A:',a,'\n'])) 
                q = ''
                a = ''
                aa= False
                qq = False

            starting = re.findall(r'^.*:',line)[0].split(':')[0].lower()
            if starting in names_5:
                q = ''.join(line.split(':')[1:]).lower()
                qq = True
                aa = False
            if starting == 'chandler' and len(q)>0:
                a=''.join(line.split(':')[1:]).lower()
                aa = True
                qq = False
                
        #else just add to whichever string we have started already
        else:
            if qq == True:
                q=q+' ' +line
            elif aa== True:
                a=a+ ' '+line
                
    #append last set of dialogues            
    if len(a)>0 and len(q)>0:
        a = re.sub("[\(\[].*?[\)\]]", "", a)
        q = re.sub("[\(\[].*?[\)\]]", "", q)
        f.write(' '.join(['Q:',q,'\n']))
        f.write(' '.join(['A:',a,'\n']))             

    f.close()
               
        
            
            
